In [1]:
import os
from functools import partial

import gymnasium as gym
import numpy as np
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import HerReplayBuffer, DDPG, DQN, SAC, TD3
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy
from stable_baselines3.common.envs import BitFlippingEnv

In [2]:
from stable_baselines3.common.callbacks import BaseCallback

class ReportRewardCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(ReportRewardCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Check if 'infos' is in locals and if 'episode' is in any of the info dicts
        if self.locals.get("infos") is not None:
            episode_rewards = [info["episode"]["r"] for info in self.locals["infos"] if "episode" in info]
            if episode_rewards:
                episode_reward_mean = np.mean(episode_rewards)
                tune.report(episode_reward_mean=episode_reward_mean)
        return True


In [3]:
env_id = 'FetchPickAndPlace-v2'
num_cpu = 4

from stable_baselines3.common.env_util import make_vec_env

vec_env = make_vec_env(env_id, n_envs=num_cpu) #vec_env_cls=SubprocVecEnv)


In [4]:
def train_sac(config, checkpoint_dir=None, data_dir=None):
    #vec_env = make_vec_env(env_id, n_envs=num_cpu, vec_env_cls=SubprocVecEnv)env = DummyVecEnv([lambda: gym.make("Pendulum-v1")])
    goal_selection_strategy = "future"
    model = SAC(
        "MultiInputPolicy",
        vec_env,
        learning_rate=config["lr"],
        replay_buffer_class=HerReplayBuffer,
        # Parameters for HER
        replay_buffer_kwargs=dict(
            n_sampled_goal=4,
            goal_selection_strategy=goal_selection_strategy,
        ),
        buffer_size=config["buffer_size"],
        learning_starts=config["learning_starts"],
        train_freq=config["train_freq"],
        gradient_steps=config["gradient_steps"],
        ent_coef=config["ent_coef"],
        tau=config["tau"],
        verbose=1
    )

    if checkpoint_dir:
        model_path = os.path.join(checkpoint_dir, "model.pkl")
        model.load(model_path)

    eval_env = vec_env#Monitor(DummyVecEnv([lambda: gym.make("Pendulum-v1")]))#eval_env = DummyVecEnv([lambda: gym.make("Pendulum-v1")])
    #report_reward_callback = ReportRewardCallback()
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path="./logs/",
        log_path="./logs/",
        eval_freq=5000,
        deterministic=True,
        render=False,
        callback_on_new_best=ReportRewardCallback()
    )

    model.learn(
        total_timesteps=100000, callback=eval_callback
    )

    if checkpoint_dir:
        model.save(os.path.join(checkpoint_dir, "checkpoint"))
        np.save(os.path.join(checkpoint_dir, "results.npy"), np.array([1.0]))

In [ ]:
if __name__ == "__main__":
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "buffer_size": tune.choice([50000, 100000, 200000]),
        "learning_starts": tune.choice([0, 1000, 10000]),
        "train_freq": tune.choice([1, 10, 100]),
        "gradient_steps": tune.choice([1, 10, 100]),
        "ent_coef": tune.loguniform(1e-4, 1e-1),
        "tau": tune.uniform(0.005, 0.02),
    }

    scheduler = ASHAScheduler(
        metric="episode_reward_mean",
        mode="max",
        max_t=100000,
        grace_period=10000,
        reduction_factor=2,
    )

    reporter = CLIReporter(
        metric_columns=["episode_reward_mean", "training_iteration"]
    )

    result = tune.run(
        partial(train_sac, data_dir="/tmp"),
        resources_per_trial={"cpu": 1},
        config=config,
        num_samples=10,
        scheduler=scheduler,
        progress_reporter=reporter,
    )

    best_trial = result.get_best_trial("episode_reward_mean", "max", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final reward: {best_trial.last_result['episode_reward_mean']}")


2023-08-16 16:29:37,321	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-08-16 16:29:38,329	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-08-16 16:29:38,362	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/tune/trainable/function_trainable.py:624: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/late

== Status ==
Current time: 2023-08-16 16:29:43 (running for 00:00:05.28)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80000.000: None | Iter 40000.000: None | Iter 20000.000: None | Iter 10000.000: None
Logical resource usage: 10.0/20 CPUs, 0/0 GPUs
Result logdir: /home/cocp5/ray_results/train_sac_2023-08-16_16-29-38
Number of trials: 10/10 (10 PENDING)
+-----------------------+----------+-------+---------------+-------------+------------------+-------------------+-------------+------------+--------------+
| Trial name            | status   | loc   |   buffer_size |    ent_coef |   gradient_steps |   learning_starts |          lr |        tau |   train_freq |
|-----------------------+----------+-------+---------------+-------------+------------------+-------------------+-------------+------------+--------------|
| train_sac_b64c9_00000 | PENDING  |       |        200000 | 0.000117486 |                1 |             10000 | 0.000505342 | 0.00753809 |           10 |
| train_sac_b64

2023-08-16 16:29:52,023	ERROR tune_controller.py:911 -- Trial task failed for trial train_sac_b64c9_00005
Traceback (most recent call last):
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/worker.py", line 2493, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=57767, ip=172.21.2.236, actor_id=3d2f6f364c793b3d1aa7571601000000, repr=func)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/

Trial name
train_sac_b64c9_00003
train_sac_b64c9_00005
train_sac_b64c9_00007


2023-08-16 16:29:52,090	ERROR tune_controller.py:911 -- Trial task failed for trial train_sac_b64c9_00003
Traceback (most recent call last):
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/_private/worker.py", line 2493, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=57765, ip=172.21.2.236, actor_id=92cb131a3ca234887619fe9901000000, repr=func)
  File "/home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/

...

== Status ==
Current time: 2023-08-16 16:30:34 (running for 00:00:55.80)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 80000.000: None | Iter 40000.000: None | Iter 20000.000: None | Iter 10000.000: None
Logical resource usage: 7.0/20 CPUs, 0/0 GPUs
Result logdir: /home/cocp5/ray_results/train_sac_2023-08-16_16-29-38
Number of trials: 10/10 (3 ERROR, 7 RUNNING)
+-----------------------+----------+--------------------+---------------+-------------+------------------+-------------------+-------------+------------+--------------+
| Trial name            | status   | loc                |   buffer_size |    ent_coef |   gradient_steps |   learning_starts |          lr |        tau |   train_freq |
|-----------------------+----------+--------------------+---------------+-------------+------------------+-------------------+-------------+------------+--------------|
| train_sac_b64c9_00000 | RUNNING  | 172.21.2.236:57762 |        200000 | 0.000117486 |                1 |             10000 

(func pid=57764) /home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.compute_reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.compute_reward` for environment variables or `env.get_attr('compute_reward')` that will search the reminding wrappers. [repeated 2x across cluster]
(func pid=57764)   logger.warn( [repeated 2x across cluster]


(func pid=57763) ------------------------------------ [repeated 2x across cluster]
(func pid=57764) --------------------------------- [repeated 74x across cluster]
(func pid=57764) | rollout/           |          | [repeated 37x across cluster]
(func pid=57764) |    ep_len_mean     | 50       | [repeated 37x across cluster]
(func pid=57764) |    ep_rew_mean     | -48      | [repeated 37x across cluster]
(func pid=57764) |    success_rate    | 0.04     | [repeated 38x across cluster]
(func pid=57764) | time/              |          | [repeated 38x across cluster]
(func pid=57764) |    episodes        | 424      | [repeated 37x across cluster]
(func pid=57764) |    fps             | 415      | [repeated 37x across cluster]
(func pid=57764) |    time_elapsed    | 51       | [repeated 37x across cluster]
(func pid=57764) |    total_timesteps | 21200    | [repeated 38x across cluster]
== Status ==
Current time: 2023-08-16 16:30:44 (running for 00:01:05.88)
Using AsyncHyperBand: num_stopped=

(func pid=57766) /home/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.compute_reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.compute_reward` for environment variables or `env.get_attr('compute_reward')` that will search the reminding wrappers.
(func pid=57766)   logger.warn(


(func pid=57766) Eval num_timesteps=20000, episode_reward=-50.00 +/- 0.00
(func pid=57766) Episode length: 50.00 +/- 0.00
(func pid=57766) Success rate: 0.00%
(func pid=57766) | eval/              |          |
(func pid=57766) |    mean_ep_length  | 50       |
(func pid=57766) |    mean_reward     | -50      |
(func pid=57764) --------------------------------- [repeated 58x across cluster]
(func pid=57764) | rollout/           |          | [repeated 28x across cluster]
(func pid=57764) |    ep_len_mean     | 50       | [repeated 28x across cluster]
(func pid=57764) |    ep_rew_mean     | -46      | [repeated 28x across cluster]
(func pid=57764) |    success_rate    | 0.08     | [repeated 29x across cluster]
(func pid=57764) | time/              |          | [repeated 29x across cluster]
(func pid=57764) |    episodes        | 756      | [repeated 28x across cluster]
(func pid=57764) |    fps             | 411      | [repeated 28x across cluster]
(func pid=57764) |    time_elapsed    | 